# Data Analysis Assignment 4
**Group: Ohm_Squad**

**Members: Rauch,Bilijesko,Frizberg**

**Datasets: Westermo**

## Initial Setup

In [5]:
# Initial setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
import os

# Configure plotting
plt.rcParams.update({
    'figure.figsize': [12, 8],
    'figure.dpi': 150,
    'figure.autolayout': True,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'font.size': 12
})

pathRaw = "./data_raw/"
pathFilter = "./data_filtered/"
pathProcessd = "./data_processed/"
pathVisuRaw = "./visu_raw/"

files = [f"system-{number}.csv" for number in range(1, 20)]

# Systems 3, 5, 6, 8, 11 and 17 do not have sys-thermal readings ! 3/5/6 -> crashes 8/11/17 -> no thermal
remove_entries = [7,10,16]
files = [item for index, item in enumerate(files) if index not in remove_entries]
files = files [0:2]

sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

np.random.seed(42)

## Loading and Filtering
Files are fetched from directory and prefiltering for columns of interst.

Processing timestamps to datetime for usage in timeseries (and usability).

Done via a function to execute for every file separately and be able to pipe if necessary.

Returning the dataframe could be either dropped or caught by either a container or piped into the next function.

In [ ]:
def load_system_data(file_dir: str, file_name: str) -> pd.DataFrame :
    """Load and prepare test system performance data.
    
    Parameters
    ----------
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    Additional outputs
    saves filtered data into dir "./data_filtered"
    
    Returns
    -------
    pd.DataFrame
        Raw dataframe with columns:
        - datetime (index)
        - load-15m
        - memory_used_pct
        - cpu-user
        - cpu-system
        - sys-thermal
        - sys-interrupt-rate
        - server-up
        - disk-io-time
    """
    file_path = file_dir + file_name

    df = pd.read_csv(file_path, delimiter = ",",usecols=["timestamp",
                                                         "load-15m",
                                                         "sys-mem-available",
                                                         "sys-mem-total",
                                                         "cpu-user",
                                                         "cpu-system",
                                                         "sys-thermal",
                                                         "sys-interrupt-rate",
                                                         "server-up",
                                                         "disk-io-time"]) # Read in data with columns
    

    
    df['datetime'] = pd.to_datetime(df['timestamp'], unit = 's', errors = 'coerce') # Create datetime from timestamp
    
    df.set_index('datetime', inplace=True) # Set datetime as index

    df['memory_used_pct'] = (1 - df['sys-mem-available']/df['sys-mem-total']) * 100 # Memory usage calculation
    df.drop(["timestamp","sys-mem-available","sys-mem-total"], axis=1, inplace=True) # Drop unneccessary data
    
    df.to_csv(pathFilter+file_name, index=True)
    
    df.describe().to_csv(f'{pathVisuRaw}{file_name}_desciption.csv')
    
    return df

# testing df = load_system_data(pathRaw,"system-3.csv")

In [ ]:
for i in files:
    this = load_system_data(pathRaw, i)

## Visualizing Raw

First: Heler functions for interacting with images and os to delete temporary files.

Second: Main function for visualizing

In [6]:
# adapted https://stackoverflow.com/questions/6996603/how-can-i-delete-a-file-or-folder-in-python
def delete_images(files: list[str]):
    """Deletes the files specified in the list of file paths.
    Parameters
    ----------
    files: list[str]
        List of names of image files to put into .pdf file. 
    
    Additional output
    ----------
        Deltes list of images.

    Returns
    -------
        None
    """
    
    for file in files:
        try:
            if os.path.exists(file):
                os.remove(file)
                #print(f"Deleted: {file}")
            else:
                print(f"File not found: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")
            
# adapted https://stackoverflow.com/questions/40906463/png-images-to-one-pdf-in-python 
# and https://www.geeksforgeeks.org/save-multiple-matplotlib-figures-in-single-pdf-file-using-python/ 
def save_image(image_names: list[str], out_dir: str, filename: str): 
    """Gathers multiple plt.figure obejcts and outputs thm into a pdf 
    
    Parameters
    ----------
    image_names: list[str]
        List of names of image files to put into .pdf file   
    out_dir: str
        Path to the directory of output .pdf file
    filename: str
        Name of output .pdf file
        
    Additional output
    ----------
        Saves a .pdf created by multiple .pngs into specified directory

    Returns
    -------
        None
    """
    image_list = [] #contains opened files
    for name in image_names:
        print(name)
        image_list.append(Image.open(name))

    image_list[0].save(f"{out_dir}{filename}.pdf", save_all=True, append_images=image_list[1:])
    for image in image_list:
        image.close()
    print(f"{out_dir}{filename}_allPlots.pdf")
    delete_images(image_names)

In [11]:

def visu_raw_data(show_plots: bool, file_dir: str, file_name: str, df_arg: pd.DataFrame = None):
    """Load and visualize filtered test system performance data.
    
    Parameters
    ----------
    show_plots: bool
        Just output files or display in notebook
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    
    optional
    df_arg: pd.DataFrame
        output from load_system_data()

    Additional outputs
    saves visualized data into dir "./visu_raw" by calling save_image() and cleaning temp-files with delete_images()
    
    Returns
    -------
        None
    """
    # Check DataFrame was passed
    if isinstance(df_arg, pd.DataFrame):
        df = df_arg
        # File name and path -> pd used => no identifier => using "./" 
        out_dir = "./"
        out_name = "Visu_output_noident"
        print("Function called with a DataFrame.")
    else:
        # Attempt to read the DataFrame from file
        try:
            file_path = file_dir + file_name
            df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
            print(f"Function called with a file: {file_path}")
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)
            # File name and path -> path used => use identifier 
            out_dir = pathVisuRaw
            out_name = file_name.replace('.csv', '')
        except Exception as e:
            print(f"Error loading the file: {e}")
            return None
    measurements = {
        "load-15m": ('load-15m', '%'),
        "memory_used_pct": ('memory_used_pct', '%'),
        "cpu-user": ('cpu-user', 'delta-s'),
        "cpu-system": ('cpu-system', 'delta-s'),
        "sys-thermal": ('sys-thermal', 'avg delta-°C/min'),
        "sys-interrupt-rate": ('sys-interrupt-rate', 'delta-s'),
        "disk-io-time": ('disk-io-time', 'delta-s')
        #,"server-up": ('server-sup', '')
    }
    image_names = []
    image_nr = 0
    
    # Plot 1: Time-Series
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle('Tme-Series - Raw Data', fontsize=16, y=1.02)

    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2

        df.iloc[::10].pivot(columns='server-up', values=measure).plot(
            ax=axes[row, col],
            legend=True, 
            alpha=0.7, 
            linewidth=2,
            color=['red','blue'],
            xlabel='Datetime',
            ylabel=f'{title} ({unit})',
            title=f'Time-Series of {measure.upper()}'
        )

    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 2: Daily Patterns
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle('Daily Patterns of Raw Measurements - mean & std ', fontsize=16, y=1.02)

    # Create hour column for grouping
    df_hour = df.copy()
    df_hour['hour'] = df_hour.index.hour

    
    for i, measurement in enumerate(measurements):
        row = i // 2
        col = i % 2
        
        # Calculate hourly statistics
        hourly_stats = df_hour.groupby('hour')[measurement].agg(['mean', 'std'])
        
        # Plot mean with standard deviation
        axes[row, col].plot(hourly_stats.index, hourly_stats['mean'], 
                        'b-', label='Mean')
        axes[row, col].fill_between(
            hourly_stats.index,
            hourly_stats['mean'] - hourly_stats['std'],
            hourly_stats['mean'] + hourly_stats['std'],
            alpha=0.2,
            label='±1 std'
        )
        
        axes[row, col].set_title(f'Daily {measurement.capitalize()} Pattern')
        axes[row, col].set_xlabel('Hour of Day')
        axes[row, col].set_ylabel(measurement)
        axes[row, col].grid(True)
        axes[row, col].legend()


    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 3: Hour-wise Distributions
    fig, axes = plt.subplots(4, 2, figsize=(15, 30))
    fig.suptitle('Measurement Distributions by Hour - Boxplots', fontsize=16, y=1.02)
        
    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2
        
        df_hour.boxplot(
            ax=axes[row, col],
            column=measure,
            by='hour'
        )
        axes[row, col].set_title(f'Daily Pattern of {title} ')
        axes[row, col].set_xlabel('Hour of Day')
        axes[row, col].set_ylabel(f'{title} ({unit})')
        axes[row, col].grid(True)

    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 4 Histograms - Distribution
    fig, axes = plt.subplots(4,2, figsize = (15, 12))
    fig.suptitle('Sensor Raw Measurements Distributions', fontsize = 14)

    for i,(measure, (title, unit)) in enumerate(measurements.items()):
        row = i // 2
        col = i % 2
        bin_num = 150
        axes[row, col].hist(df[measure], bins = bin_num, density = True, alpha = 0.7)

        counts, bins = np.histogram(df[measure], bins = bin_num)
        bin_centers = (bins[:-1] + bins [1:]) / 2
        axes[row, col].plot(bin_centers, counts/counts.sum(), 'r-', lw = 2, label = 'Distribution')        
        
        axes[row, col].set_title(f'Distribution of {title} ')
        axes[row, col].set_xlabel( f'{title} ({unit})')
        axes[row, col].set_ylabel('Share')
        axes[row, col].grid(True)
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    # Plot 5: Correlation Analysis
    fig, (ax) = plt.subplots(1, 1, figsize=(15, 15))
    fig.suptitle('Correlation Analysis - of Raw Measurements Correlations', y=1.02, fontsize=16)

    # Original correlations
    sns.heatmap(
        df[measurements.keys()].corr(),
        annot=True,
        cmap='coolwarm',
        center=0,
        fmt='.2f',
        ax=ax
    )
    
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    ' !!! SOURCE !!!'
    # Plot 6 Hexbins
    measure = list(measurements.keys())
    pairs = [(measure[i], measure[j]) for i in range(len(measure)) for j in range(i + 1, len(measure))]

    # Number of subplots
    n_rows = 7
    n_cols = 3

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    fig.suptitle('Hexbins of Raw Measurements', y=1.02, fontsize=16)
    axes = axes.flatten()  # Flatten the axes to make indexing easier

    # Loop over the pairs
    for i, (measure1, measure2) in enumerate(pairs):
        ax = axes[i]
        x = df[measure1]
        y = df[measure2]
        
        # Plot hexbin
        hb = ax.hexbin(x, y, gridsize=30, cmap='viridis')
        ax.set_xlabel(measure1)
        ax.set_ylabel(measure2)
        ax.set_title(f'Hexbin: {measure1} vs {measure2}')
        # Add color bar
        fig.colorbar(hb, ax=ax)
        
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------

    # Plot 7: Scatter Matrix
    # Get data without duplicates by taking mean for each timestamp
    df_plot = df.groupby(df.index)[measure].mean()
    try:
        pp = sns.pairplot(data=df_plot,
                            diag_kind='kde',
                            plot_kws={'alpha': 0.5, 's': 20},
                            height = 6)
    except Exception as e:
        print(f"Warning: Could not create scatter matrix plot: {str(e)}")

    fig = pp.figure
    fig.suptitle('Scatter Matrix of Raw Measurements', y=1.02, fontsize=16)
    
    #----------------------------------------------
    plt.tight_layout()
    
    temp_name = f"{out_dir}{out_name}_plot_{image_nr}.png"
    fig.savefig(temp_name, dpi=200, bbox_inches='tight')
    image_names.append(temp_name)
    image_nr += 1
    #----------------------------------------------
    
    #save_image(image_names, out_dir, out_name)
    if not show_plots:
        plt.close("all")


# testing visu_raw_data(True, None, None,df)

In [12]:
for file in files:
    visu_raw_data(False, pathFilter,file,None)
    plt.close("all") #for safety

Function called with a file: ./data_filtered/system-1.csv
Function called with a file: ./data_filtered/system-2.csv


Processing

In [ ]:
#TODO

Visualize Raw & Processed

In [ ]:
def preprocess_system_data(show_plots: bool, file_dir: str, file_name: str, df_arg: pd.DataFrame = None):
    """Preprocess system performance data.
    Cleans data with:
          * Invalid values removed
          * Duplicates handled
          * Outliers removed
          * Missing values interpolated
    Parameters
    ----------
    show_plots: bool
        Just output files or display in notebook
    file_dir : str
        Path to the CSV data file location (directory)
    file_name : str
        Name of the specified CSV file
    
    
    optional
    df_arg: pd.DataFrame
        output from load_system_data()

    Additional outputs
    saves visualized data into dir "./visu_raw" by calling save_image() and cleaning temp-files with delete_images()
    
    Returns
    -------
        None
    """
    
    #TODO !! -----------------------
    
    # Check DataFrame was passed
    if isinstance(df_arg, pd.DataFrame):
        df = df_arg
        # File name and path -> pd used => no identifier => using "./" 
        out_dir = "./"
        out_name = "Visu_output_noident"
        print("Function called with a DataFrame.")
    else:
        # Attempt to read the DataFrame from file
        try:
            file_path = file_dir + file_name
            df = pd.read_csv(file_path, delimiter = ",",usecols=["datetime","load-15m","memory_used_pct","cpu-user","cpu-system","sys-thermal","sys-interrupt-rate","server-up","disk-io-time"])
            print(f"Function called with a file: {file_path}")
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)
            # File name and path -> path used => use identifier 
            out_dir = pathVisuRaw
            out_name = file_name.replace('.csv', '')
        except Exception as e:
            print(f"Error loading the file: {e}")
            return None
    measurements = {
        "load-15m": ('load-15m', '%'),
        "memory_used_pct": ('memory_used_pct', '%'),
        "cpu-user": ('cpu-user', 'delta-s'),
        "cpu-system": ('cpu-system', 'delta-s'),
        "sys-thermal": ('sys-thermal', 'avg delta-°C/min'),
        "sys-interrupt-rate": ('sys-interrupt-rate', 'delta-s'),
        "disk-io-time": ('disk-io-time', 'delta-s')
        #,"server-up": ('server-sup', '')
    }

    # Store original data
    df_original = df.copy()
   
   
    # Define valid ranges
    valid_ranges = {
        'load-15m': (0, 0.5),           # System load
        'memory_used_pct': (0, 100),   # Percentage
        'cpu-user': (0.0, 2.0),            # Rate of change in CPU time
        'sys-thermal': (-10, 10),      # Rate of change in °C
        'server-up': (0, float('inf')) # Server availability
    }
    
    # YOUR CODE HERE
    # 1. Handle invalid values

    for column, (min_val, max_val) in valid_ranges.items():
        invalid_mask = (df[column] < min_val) | (df[column] > max_val)
        print(f"Removing {invalid_mask.sum()} invalid values from {column}")
        df.loc[invalid_mask, column] = np.nan
    
    # 2. Handle duplicates
    print("Handling duplicate timestamps...")
    df = df.groupby(['datetime', 'server-up']).agg({
        'load-15m': 'mean',
        'memory_used_pct': 'mean',
        'cpu-user': 'mean',
        'sys-thermal': 'mean'
    }).reset_index()
    
    # 3. Remove outliers
    def remove_outliers_iqr(df, column):
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        valid_mask = (df[column] >= Q1 - 1.5*IQR) & (df[column] <= Q3 + 1.5*IQR)
        invalid_count = (~valid_mask).sum()
        print(f"Removing {invalid_count} outliers from {column}")
        return df[column].where(valid_mask, np.nan)
    
    for column in ['load-15m', 'memory_used_pct', 'cpu-user', 'sys-thermal']:
        df[column] = remove_outliers_iqr(df, column)
    
    # 4. Handle missing values

    print("\nHandling missing values...")
    print(f"Missing values before handling: \n{df.isnull().sum()}")
    
    # Handle missing values by sensor
    df_cleaned = pd.DataFrame()
    for sensor in sorted(df['server-up'].unique()):
        print(f"Processing sensor {sensor}...")
        sensor_data = df[df['server-up'] == sensor].copy()
        
        # Ensure datetime column is a DatetimeIndex
        sensor_data['datetime'] = pd.to_datetime(sensor_data['datetime'])
        sensor_data.set_index('datetime', inplace=True)
        
        # Resample to regular intervals (e.g., 5-minute intervals)
        sensor_data = sensor_data.resample('5min').mean() # 5T->5min
        
        # Interpolate missing values
        for column in ['load-15m', 'memory_used_pct', 'cpu-user', 'sys-thermal']:
            sensor_data[column] = sensor_data[column].interpolate(
                method='linear',
                limit=4 
            )

    # Add back the sensor ID
        sensor_data['server-up'] = sensor
        
        # Append to cleaned dataframe
        df_cleaned = pd.concat([df_cleaned, sensor_data], sort=False)
    
    # Sort by datetime
    df_cleaned.sort_index(inplace=True)
    
    print(f"Missing values after handling: \n{df_cleaned.isnull().sum()}")
    
    print(f"\nOriginal shape: {df_original.shape}")
    print(f"Cleaned shape: {df_cleaned.shape}")
    
    

    return df_cleaned, df_original
    
